In [ ]:
#import library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
#홀로그램 검출
def get_holo(img, coordinate):

  #img = img경로
  if (coordinate[0]>coordinate[1] or coordinate[2]>coordinate[3]):
    print("x1(y1) 좌표보다 x2(y2) 좌표값이 더 큽니다.")
    return

  #ROI = img[80:430, 80:430] #홀로그램에서 태극마크 부분
  ROI = img[coordinate[0]:coordinate[1], coordinate[2]:coordinate[3]] #홀로그램에서 태극마크 부분
  ROI_size = (coordinate[1]-coordinate[0]) * (coordinate[3]-coordinate[2])
  
  cv2.imshow(ROI)
  HSV = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV) #BGR to HSV
  H, S, V = cv2.split(HSV)
  #H = cv2.equalizeHist(H)
  hist = cv2.calcHist(H, [0], None, [181], [0, 181])
  plt.plot(hist)
  plt.show()

  _, H_bin = cv2.threshold(H, 150, 255, cv2.THRESH_TOZERO_INV) #H 채널에서 홀로그램 판별
  _, H_bin1 = cv2.threshold(H_bin, 30, 255, cv2.THRESH_BINARY)
  cv2.imshow(H_bin1)

  holo_size = (sum((H_bin1/255).flatten()))
  holo_sore = round(holo_size/ROI_size * 100, 2)
  
  return holo_sore

In [ ]:
#모서리 좌표 정렬
def reorderPts(pts):
  idx = np.lexsort((pts[:, 1], pts[:, 0])) # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
  pts = pts[idx] # x좌표로 정렬
  if pts[0, 1] > pts[1, 1]:
    pts[[0, 1]] = pts[[1, 0]]
  if pts[2, 1] < pts[3, 1]:
    pts[[2, 3]] = pts[[3, 2]]
  return pts

In [ ]:
#민증영역 추출
def crob_contours(img):
  img = cv2.imread(img, cv2.IMREAD_COLOR)
  src = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
  src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

  h, w = src.shape[:2]
  dst1 = np.zeros((h, w, 3), np.uint8)
  dst2 = np.zeros((h, w, 3), np.uint8)

  dw, dh = 860, 540 
  srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32)
  dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32)
  dst = np.zeros((dh, dw), np.uint8)

  # 이진화
  _, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  # 외곽선 검출
  contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  for i in range(len(contours)):
      pts = contours[i]

      c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
      cv2.drawContours(dst1, contours, i, c, 1)

      # 너무 작은 객체는 제외
      if (cv2.contourArea(pts) < 1000):
          continue

      # 외곽선 근사화
      approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True)*0.02, True)
      
      # 컨벡스가 아니면 제외
      if not cv2.isContourConvex(approx):
          continue

      
      if len(approx) == 4:
          cv2.drawContours(dst2, contours, i, c, 2)
          srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))

          pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
          dst = cv2.warpPerspective(src, pers, (dw, dh), flags=cv2.INTER_CUBIC)
          dst_rgb = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
          

  return dst

In [ ]:
# 최종main 함수
def get_hologram(img, coordinate = [80, 430, 80, 430]):
  """
  img: 이미지 파일 경로
  coordinate: ROI 좌표값, 기본값 [80, 430, 80, 430]은 태극무늬 홀로그램 필름
  """

  dst = crob_contours(img)
  holo_score = get_holo(dst, coordinate)
  is_hologram = "Y" if (15<=holo_score and holo_score<=40) else "N" #빛으로 인해 holo_score가 높게 잡힐 경우 재 촬영 필요
  if (is_hologram=="N"): print("신분증을 다시 찍어주세요.")
  return is_hologram , holo_score